# Load in data
Currently supported datasets: power, protein, kin8nm

In [1]:
import torch 
import geometric_kernels.torch 

torch.set_default_dtype(torch.float64)

In [2]:
import os
import pandas as pd 
import torch 
from torch import Tensor
from torch.utils.data import TensorDataset, DataLoader, Dataset


class UCIDataset:

    UCI_BASE_URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/'

    def __init__(self, name: str, path: str = '../../data/uci/', normalize: bool = True, seed: int | None = None): 
        self.name = name 
        self.path = path 
        self.csv_path = os.path.join(self.path, self.name + '.csv')

        # Set generator if seed is provided.
        self.generator = torch.Generator()
        if seed is not None: 
            self.generator.manual_seed(seed)

        # Load, shuffle, split, and normalize data. TODO except for load, these don't need to be object methods. 
        # We keep the standard deviation of the test set for log-likelihood evaluation.
        x, y = self.load_data()
        x, y = self.shuffle(x, y, generator=self.generator)     
        self.train_x, self.train_y, self.test_x, self.test_y = self.split(x, y)
        self.test_y_std = self.test_y.std(dim=0, keepdim=True)
        self.train_x, self.train_y, self.test_x, self.test_y = map(
            self.normalize, (self.train_x, self.train_y, self.test_x, self.test_y))

    @property
    def dimension(self) -> int:
        return self.train_x.shape[-1]

    @property 
    def train_dataset(self) -> Dataset:
        return TensorDataset(self.train_x, self.train_y)
    
    @property
    def test_dataset(self) -> Dataset:
        return TensorDataset(self.test_x, self.test_y)

    def read_data(self) -> tuple[Tensor, Tensor]:
        xy = torch.from_numpy(pd.read_csv(self.csv_path).values)
        return xy[:, :-1], xy[:, -1]

    def download_data(self) -> None:
        NotImplementedError

    def load_data(self, overwrite: bool = False) -> tuple[Tensor, Tensor]:
        if overwrite or not os.path.isfile(self.csv_path):
            self.download_data()
        return self.read_data()

    def normalize(self, x: Tensor) -> Tensor:
        return (x - x.mean(dim=0)) / x.std(dim=0, keepdim=True)
    
    def shuffle(self, x: Tensor, y: Tensor, generator: torch.Generator) -> tuple[Tensor, Tensor]:
        perm_idx = torch.randperm(x.size(0), generator=generator)
        return x[perm_idx], y[perm_idx]
    
    def split(self, x: Tensor, y: Tensor, test_size: float = 0.1) -> tuple[Tensor, Tensor, Tensor, Tensor]: 
        """
        Split the dataset into train and test sets.
        """
        split_idx = int(test_size * x.size(0))
        return x[split_idx:], y[split_idx:], x[:split_idx], y[:split_idx]


class Kin8mn(UCIDataset):

    DEFAULT_URL = 'https://raw.githubusercontent.com/liusiyan/UQnet/master/datasets/UCI_datasets/kin8nm/dataset_2175_kin8nm.csv'

    def __init__(self, path: str = '../../data/uci/', normalize: bool = True, seed: int | None = None, url: str = DEFAULT_URL):
        super().__init__(name='kin8nm', path=path, normalize=normalize, seed=seed)
        self.url = url 

    def download_data(self) -> None:
        df = pd.read_csv(self.url)
        os.makedirs(self.path, exist_ok=True)
        df.to_csv(self.csv_path, index=False)

# Instantiate model

This is done using the same model arguments as for the benchmarking experiment

In [3]:
from linear_operator.operators import DiagLinearOperator
from gpytorch.distributions import MultivariateNormal


class SphereProjector(torch.nn.Module):
    def __init__(self):
        super().__init__()
        b = 1. + torch.exp(torch.randn(torch.Size([])))
        self.register_parameter('b', torch.nn.Parameter(b))
        self.norm = None 

    def forward(self, x: Tensor, y: Tensor | None = None) -> tuple[Tensor, Tensor] | Tensor:
        b = self.b.expand(*x.shape[:-1], 1)
        x_cat_b = torch.cat([x, b], dim=-1)
        self.norm = x_cat_b.norm(dim=-1, keepdim=True)
        if y is None:
            return x_cat_b / self.norm
        else:
            return x_cat_b / self.norm, y / self.norm.squeeze(-1)
    
    def inverse(self, mvn: MultivariateNormal) -> MultivariateNormal:
        L = DiagLinearOperator(self.norm.squeeze(-1))
        mean = mvn.mean @ L
        cov = L @ mvn.lazy_covariance_matrix @ L
        return MultivariateNormal(mean=mean, covariance_matrix=cov)

In [4]:
from torch import Tensor 
from gpytorch.means import Mean
from gpytorch.kernels import ScaleKernel
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from mdgp.kernels import GeometricMaternKernel


import torch 
from gpytorch import Module, settings
from gpytorch.distributions import MultivariateNormal
from gpytorch.utils.memoize import cached, clear_cache_hook
from linear_operator.operators import DiagLinearOperator
from functools import cached_property
# TODO Maybe just move the functions from spherical_harmonic_features.py into this file?
from mdgp.utils.spherical_harmonic_features import num_spherical_harmonics_to_degree, matern_Kuu, matern_LT_Phi


class SphericalHarmonicFeaturesVariationalStrategy(Module):
    def __init__(
        self,
        model: ApproximateGP,
        variational_distribution: CholeskyVariationalDistribution,
        dimension: int, 
        num_spherical_harmonics: int, 
        jitter_val: float | None = None,
    ):
        super().__init__()
        self._jitter_val = jitter_val

        # model, set via object.__setattr__ to avoid treatment as a module, parameter, or buffer
        object.__setattr__(self, "_model", model)

        # Variational distribution
        self._variational_distribution = variational_distribution
        self.register_buffer("variational_params_initialized", torch.tensor(0))

        # spherical harmonics 
        self.dimension = dimension 
        self.degree, self.num_spherical_harmonics = num_spherical_harmonics_to_degree(num_spherical_harmonics, dimension)

    @property
    def jitter_val(self) -> float:
        if self._jitter_val is None:
            return settings.variational_cholesky_jitter.value(dtype=torch.get_default_dtype())
        return self._jitter_val

    @property
    def model(self) -> ApproximateGP:
        return self._model
    
    @property
    def covar_module(self) -> ScaleKernel | GeometricMaternKernel:
        return self.model.covar_module
    
    @cached_property
    def base_kernel(self) -> GeometricMaternKernel:
        if isinstance(self.covar_module, ScaleKernel):
            return self.covar_module.base_kernel
        else:
            return self.covar_module
    
    @property
    def mean_module(self) -> Mean:
        return self.model.mean_module

    @property
    def kappa(self) -> Tensor:
        return self.base_kernel.lengthscale
    
    @property
    def nu(self) -> Tensor | float:
        return self.base_kernel.nu
    
    @property 
    def outputscale(self) -> Tensor:
        return self.covar_module.outputscale if hasattr(self.covar_module, "outputscale") else torch.tensor(1.0)

    @property 
    def sigma(self) -> Tensor: 
        return self.outputscale.sqrt()

    def _clear_cache(self) -> None:
        clear_cache_hook(self)

    @property
    @cached(name="prior_distribution_memo")
    def prior_distribution(self) -> MultivariateNormal:
        covariance_matrix = DiagLinearOperator(torch.ones(self.num_spherical_harmonics))
        mean = torch.zeros(self.num_spherical_harmonics)
        return MultivariateNormal(mean=mean, covariance_matrix=covariance_matrix)
    
    @property 
    @cached(name="cholesky_factor_prior_memo")
    def cholesky_factor_prior(self) -> DiagLinearOperator:
        Kuu = matern_Kuu(max_ell=self.degree, d=self.dimension, kappa=self.kappa, nu=self.nu, sigma=self.sigma)
        return Kuu.cholesky() # Kuu is a DiagLinearOperator, so .cholesky() is equivalent to .sqrt() 
        
    @property
    @cached(name="variational_distribution_memo")
    def variational_distribution(self) -> MultivariateNormal:
        return self._variational_distribution()

    def forward(self, x: Tensor, **kwargs) -> MultivariateNormal:
        # inducing-inducing prior
        pu = self.prior_distribution
        invL_muu, invL_Kuu_invLt = pu.mean, pu.lazy_covariance_matrix

        # input-input prior
        px = self.model.forward(x)
        mux, Kxx = px.mean, px.lazy_covariance_matrix

        # inducing-inducing variational
        qu = self.variational_distribution
        invL_m, invL_S_invLt = qu.mean, qu.lazy_covariance_matrix

        # Add jitter to Kxx and invL_Kuu_invLt for numerical stability
        Kxx = Kxx.add_jitter(self.jitter_val)
        invL_Kuu_invLt = invL_Kuu_invLt.add_jitter(self.jitter_val)

        # inducing-input prior  
        LT_Phi = matern_LT_Phi(
            x, max_ell=self.degree, d=self.dimension, kappa=self.kappa, nu=self.nu, sigma=self.sigma,
        ) # [..., O, num_harmonics, N]
        
        # Update the mean
        mean_update = torch.einsum('...ij,...i->...j', LT_Phi, invL_m - invL_muu) # [..., O, num_harmonics, N] @ [O, num_harmonics] -> [..., O, N]
        updated_mean = mux + mean_update # [..., O, N] + [..., O, N] -> [..., O, N]

        # Update the covariance matrix
        covariance_matrix_update = LT_Phi.mT @ (invL_S_invLt - invL_Kuu_invLt) @ LT_Phi # [O, num_harmonics, num_harmonics] @ [O, num_harmonics, num_harmonics] @ [O, num_harmonics, N] -> [O, num_harmonics, N]
        updated_covariance_matrix = Kxx + covariance_matrix_update # [..., O, N, N] + [..., O, N, N] -> [..., O, N, N]

        return MultivariateNormal(mean=updated_mean, covariance_matrix=updated_covariance_matrix)

    def kl_divergence(self) -> Tensor:
        with settings.max_preconditioner_size(0):
            kl_divergence = torch.distributions.kl.kl_divergence(self.variational_distribution, self.prior_distribution)
        return kl_divergence
    
    def __call__(self, x: Tensor, prior: bool = False, **kwargs) -> MultivariateNormal:
        # If we're in prior mode, then we're done!
        if prior:
            return self.model.forward(x, **kwargs)

        # Delete previously cached items from the training distribution
        if self.training:
            self._clear_cache()

        # (Maybe) initialize variational distribution
        if not self.variational_params_initialized.item():
            prior_dist = self.prior_distribution
            self._variational_distribution.initialize_variational_distribution(prior_dist)
            self.variational_params_initialized.fill_(1)

        return super().__call__(x, **kwargs)

INFO: Using numpy backend


# Simple shallow variational GP

In [5]:
import gpytorch


class SHApproximateGP(gpytorch.models.ApproximateGP):
    def __init__(self, dimension: int, num_spherical_harmonics: int, 
                 mean: Mean, covar_module: GeometricMaternKernel, variational_distribution: CholeskyVariationalDistribution):
        variational_strategy = SphericalHarmonicFeaturesVariationalStrategy(
            model=self,
            variational_distribution=variational_distribution,
            dimension=dimension,
            num_spherical_harmonics=num_spherical_harmonics,
        )
        super().__init__(variational_strategy)
        self.mean_module = mean
        self.covar_module = covar_module
        self.likelihood = gpytorch.likelihoods.GaussianLikelihood()


    def forward(self, x: Tensor) -> MultivariateNormal:
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)
    

class IPApproximateGP(gpytorch.models.ApproximateGP):
    def __init__(self, mean: Mean, covar_module: GeometricMaternKernel, inducing_points: Tensor, variational_distribution: CholeskyVariationalDistribution):
        variational_strategy = gpytorch.variational.VariationalStrategy(
            self, 
            inducing_points=inducing_points,
            variational_distribution=variational_distribution, 
            learn_inducing_locations=False
        )
        super().__init__(variational_strategy)
        self.mean_module = mean
        self.covar_module = covar_module
        self.likelihood = gpytorch.likelihoods.GaussianLikelihood()

    def forward(self, x: Tensor) -> MultivariateNormal:
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

# Train model
Training has to be done differently from the bechmarking experiment, because we need minibatch SGD with the larger datasets and minibatch metrics. 

In [25]:
def print_smallest_eigenvalues(mvn):
    # mvn = projector.inverse(mvn)
    smallest_eigenvalues = torch.linalg.eigvalsh(mvn.lazy_covariance_matrix.evaluate()).min()
    print(f"Smallest eigenvalue: {smallest_eigenvalues.item()}")

In [26]:
from geometric_kernels.spaces import Hypersphere
from mdgp.utils.spherical_harmonic_features import num_spherical_harmonics_to_degree


dataset = Kin8mn()

# Generic parameters
num_spherical_harmonics = 50
dimension = dataset.dimension + 1

degree, num_spherical_harmonics = num_spherical_harmonics_to_degree(num_spherical_harmonics, dimension)
space = Hypersphere(dimension)

batch_shape = torch.Size([])

# Model with spherical harmonic features
mean = gpytorch.means.ConstantMean()
base_kernel = GeometricMaternKernel(nu=2.5, space=space, num_eigenfunctions=4, batch_shape=batch_shape)
covar_module = ScaleKernel(base_kernel)
variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
    num_inducing_points=num_spherical_harmonics, batch_shape=batch_shape
)

model_sh = SHApproximateGP(
    dimension=dimension,
    num_spherical_harmonics=num_spherical_harmonics,
    mean=mean,
    covar_module=covar_module,
    variational_distribution=variational_distribution
)

# Model with inducing points 
mean = gpytorch.means.ConstantMean()
base_kernel = GeometricMaternKernel(nu=2.5, space=space, num_eigenfunctions=4, batch_shape=batch_shape)
covar_module = ScaleKernel(base_kernel)
variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
    num_inducing_points=num_spherical_harmonics, batch_shape=batch_shape
)

inducing_points = torch.randn(num_spherical_harmonics, dimension)
inducing_points = inducing_points / inducing_points.norm(dim=-1, keepdim=True)
model_ip = IPApproximateGP(
    mean=mean,
    covar_module=covar_module,
    inducing_points=inducing_points,
    variational_distribution=variational_distribution
)


# Arbitrary projector 
projector = SphereProjector()

The number of spherical harmonics requested does not lead to complete levels of spherical harmonics. We have thus increased the number to 54, which includes all spherical harmonics up to degree 3 (incl.)


In [27]:
model = model_sh

batch_size = 256
train_loader = DataLoader(dataset.train_dataset, batch_size=batch_size, shuffle=True)

# 
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
mll = gpytorch.mlls.VariationalELBO(model.likelihood, model, dataset.train_x.size(0))

for x_batch, y_batch in train_loader:
    x_batch, y_batch = projector(x_batch, y_batch)
    optimizer.zero_grad()
    output = model(x_batch)
    print_smallest_eigenvalues(output)
    loss = -mll(output, y_batch)
    loss.backward()
    optimizer.step()

    print(loss.item())

Smallest eigenvalue: 9.999999986902469e-07
1.2891064465375877
Smallest eigenvalue: -0.1357181145969312
1.1934144881472015
Smallest eigenvalue: -0.31078934586160606
1.0703865792531075
Smallest eigenvalue: -0.46751262150711514
0.9690412907119924
Smallest eigenvalue: -0.7028916434757697
0.8887080844428432
Smallest eigenvalue: -0.7881276725005424
0.8094053092966281
Smallest eigenvalue: -0.8454695057135702
0.733132044177425
Smallest eigenvalue: -0.8768659247019591
0.6799833529956283
Smallest eigenvalue: -1.0255301171162763
0.652940862556185
Smallest eigenvalue: -0.9627411756820207
0.6183478034115152
Smallest eigenvalue: -0.7940298870665754
0.5922879532189999
Smallest eigenvalue: -0.7649416935806412
0.5508058104598297
Smallest eigenvalue: -0.7691073318780455
0.5176412553545229
Smallest eigenvalue: -0.7322238661869614
0.4622863761150729
Smallest eigenvalue: -0.6808265091418633
0.4410889824106476
Smallest eigenvalue: -0.5731409060232125
0.3973870074592462
Smallest eigenvalue: -0.47903473454256

In [28]:
model = model_ip

batch_size = 256
train_loader = DataLoader(dataset.train_dataset, batch_size=batch_size, shuffle=True)

# 
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
mll = gpytorch.mlls.VariationalELBO(model.likelihood, model, dataset.train_x.size(0))

for x_batch, y_batch in train_loader:
    x_batch, y_batch = projector(x_batch, y_batch)
    optimizer.zero_grad()
    output = model(x_batch)
    print_smallest_eigenvalues(output)
    loss = -mll(output, y_batch)
    loss.backward()
    optimizer.step()

    print(loss.item())

Smallest eigenvalue: 9.999999955160189e-07
1.3038401902801835
Smallest eigenvalue: 9.999999971823377e-07
1.168367487019407
Smallest eigenvalue: 9.999999963133244e-07
1.016595786504766
Smallest eigenvalue: 9.999999983053191e-07
0.9079352812559105
Smallest eigenvalue: 9.999999985664043e-07
0.841934390424427
Smallest eigenvalue: 9.999999983287972e-07
0.7654001050887477
Smallest eigenvalue: 9.999999983572765e-07
0.7631627858428819
Smallest eigenvalue: 9.999999983287072e-07
0.7536802053385149
Smallest eigenvalue: 9.999999984665512e-07
0.7057247497882502
Smallest eigenvalue: 9.999999984865905e-07
0.6492769598871178
Smallest eigenvalue: 9.999999985245056e-07
0.608138439190725
Smallest eigenvalue: 9.999999985290089e-07
0.5647388357474672
Smallest eigenvalue: 9.999999985193053e-07
0.5345302676337101
Smallest eigenvalue: 9.999999986204065e-07
0.4971043280994499
Smallest eigenvalue: 9.999999985592236e-07
0.47382076199168943
Smallest eigenvalue: 9.999999985315982e-07
0.4354417100359528
Smallest ei

### Debug

In [14]:
import plotly.express as px 
import pandas as pd 


def plot_kernel_vs_angle(kernel, dimension: int, n: int = 100): 
    pole = torch.zeros(1, dimension)
    pole[:, -1] = 1.
    theta = torch.linspace(0, torch.pi, n)
    x = torch.cat([torch.zeros(n, dimension - 2), theta.cos().unsqueeze(-1), theta.sin().unsqueeze(-1)], dim=-1)
    with torch.no_grad():
        y = kernel(x).lazy_covariance_matrix[..., 0]
        if y.ndim == 2: 
            y = y.mean(0)

    data = pd.DataFrame({'theta': theta.squeeze().numpy(), 'y': y})
    fig = px.line(data, x='theta', y='y')
    fig.show()
